## Database creation and cleaning for Stack Overflow dataset
#### The current notebook needs the "posts_stackoverflow.xml" file inside the raw_dataset folder in order to run properly.
The dataset is read, parsed and exported into three CSV files (train, validation and test).
It labels the questions based on their "score" property between 1 (good) and 0 (bad) question. These labels are calculated given the median score of all questions considered for the training set.

In [ ]:
# INSTALL LIBRARIES AND DOWNLOAD FILES
# !pip install py7zr
# !wget https://archive.org/download/stackexchange/electronics.stackexchange.com.7z

In [ ]:
# UNZIP FILE
'''
import py7zr
print(py7zr.__version__)
import py7zr
with py7zr.SevenZipFile('/content/electronics.7z', mode='r') as z:
    z.extractall("/content/data")
'''

In [9]:
#Parse XML to CSV (cleaning data)
import xml.etree.ElementTree as ET
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
import random
import re

In [28]:
#Class to hold all scores in the dataset
class Scores:
    def __init__(self):
        #Array of all scores
        self.scores_list = []
        self.median = None

    def append(self, score):
        #append question to dictionary
        self.scores_list.append(score)
        
    def process(self):
        # orders scores and find median
        # call before trying to score something
        self.scores_list.sort()
        self.median = self.scores_list[len(self.scores_list)//2]
        
    #Function that returns the label given the num of likes
    def get_label(self, likes):
        return 0 if likes <= self.median else 1

In [29]:
#Create Scores objects based on all questions in the dataset
data_size = 20094655
scores = {str(year):Scores() for year in range(2010,2019+1)}
i = 0
for event, elem in ET.iterparse("../raw_dataset/posts_stackoverflow.xml"):
    #Add score if type is question
    if elem.tag == "row" and event == "end" and elem.attrib["PostTypeId"] == "1":
        year = str(elem.attrib["CreationDate"]).split("T")[0].split("-")[0]
        if int(year)>=2020:
            break
        if int(year)>=2010:
            scores[year].append(int(elem.attrib["Score"]))
            i += 1
    elem.clear()
    if i % 1000000 == 0 and i>0:
        print(i)
    if i == data_size:
        break
        
print("Processing scores...")
for scorer in scores.values():
    scorer.process()

#for scorer in scores.values():
#    scorer.process()


1000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
2000000
3000000
3000000
3000000
3000000
3000000
4000000
5000000
5000000
6000000
7000000
8000000
8000000
9000000
10000000
11000000
12000000
12000000
12000000
12000000
13000000
13000000
14000000
14000000
14000000
14000000
14000000
14000000
15000000
15000000
15000000
15000000
15000000
15000000
15000000
15000000
15000000
15000000
16000000
16000000
16000000
16000000
17000000
18000000
Processing scores...


In [33]:
data_size = 18050016
data_split = 1000000

idx_val = set(random.sample(range(data_split),int(data_split*0.05)))
idx_test = set()
for i in range(int(data_split*0.05)):
    n = random.randint(0, data_split-1)
    while n in idx_val or n in idx_test:
        n = random.randint(0, data_split-1)
    idx_test.add(n)
print(len(idx_test))
print(len(idx_val))
print(len(idx_test.intersection(idx_val)))
z = np.zeros(data_split)
z[list(idx_test)] = 1
z[list(idx_val)] = 2
print(np.count_nonzero(z == 0))
print(np.count_nonzero(z == 1))
print(np.count_nonzero(z == 2))

50000
50000
0
900000
50000
50000


In [34]:
#Parse XML to CSV (cleaning data)

#Creates list of tags given the string
def createTags(tags):
    return "|".join(tags[1:-1].split("><"))

#Clean sentence (remove non alpha chars)
def cleanSentence(sentence):
    p = re.compile(r'<.*?>')
    sentence = p.sub('', sentence) 
    sentence = ''.join([(i.lower() if i.isalpha() else " ") for i in sentence if i.isalpha() or i == " " or i == "-"])
    return sentence
    
#Opens CSV file to write parsed rows
i, j = 0, 0
with open('../processed_files/data_stackoverflow_train.csv', mode='w', newline='',encoding="utf8", buffering=1) as data_file:
    with open('../processed_files/data_stackoverflow_test.csv', mode='w', newline='',encoding="utf8", buffering=1) as data_file_test:
        with open('../processed_files/data_stackoverflow_val.csv', mode='w', newline='',encoding="utf8", buffering=1) as data_file_val:
            #Headers for train
            data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            data_writer.writerow(["stars","title","tags","score","creation_date","body"]) #write headers
            #Headers for test
            data_writer_test = csv.writer(data_file_test, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            data_writer_test.writerow(["stars","title","tags","score","creation_date","body"]) #write headers
            #Headers for val
            data_writer_val = csv.writer(data_file_val, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            data_writer_val.writerow(["stars","title","tags","score","creation_date","body"]) #write headers
            #Loop to iterate through every element on the XML file
            for event, elem in ET.iterparse("../raw_dataset/posts_stackoverflow.xml"):
                #Write row if tag is row and post type is 1 (meaning it is a question)
                if elem.tag == "row" and event == "end" and elem.attrib["PostTypeId"] == "1":
                    year = str(elem.attrib["CreationDate"]).split("T")[0].split("-")[0]
                    if int(year)>=2020:
                        break
                    if int(year)>=2010:
                        if i % (data_size//data_split) == 0:
                            score = int(elem.attrib["Score"])
                            if j in idx_test:
                                data_writer_test.writerow([scores[year].get_label(score),cleanSentence(elem.attrib["Title"]),createTags(elem.attrib["Tags"].lower()),elem.attrib["Score"],elem.attrib["CreationDate"],cleanSentence(elem.attrib["Body"])])
                            elif j in idx_val:
                                data_writer_val.writerow([scores[year].get_label(score),cleanSentence(elem.attrib["Title"]),createTags(elem.attrib["Tags"].lower()),elem.attrib["Score"],elem.attrib["CreationDate"],cleanSentence(elem.attrib["Body"])])
                            else:
                                data_writer.writerow([scores[year].get_label(score),cleanSentence(elem.attrib["Title"]),createTags(elem.attrib["Tags"].lower()),elem.attrib["Score"],elem.attrib["CreationDate"],cleanSentence(elem.attrib["Body"])])  
                            j += 1
                    i += 1
                elem.clear()
                if i % 1000000 == 0:
                    print(i)
                if j == data_split:
                    break


1000000
1000000
2000000
2000000
3000000
3000000
3000000
3000000
3000000
4000000
4000000
5000000
5000000
6000000
6000000
6000000
7000000
8000000
8000000
9000000
9000000
9000000
9000000
9000000
9000000
9000000
9000000
9000000
9000000
9000000
10000000
11000000
11000000
11000000
11000000
12000000
13000000
13000000
14000000
14000000
14000000
14000000
14000000
15000000
15000000
15000000
15000000
16000000
16000000
16000000
17000000
18000000
18000000


In [36]:
total = 0
total_true = 0
with open("../processed_files/data_stackoverflow_train.csv", "r", encoding="utf8") as f:
    reader = csv.DictReader(f, delimiter=",")
    for row in reader:
        total += 1
        total_true += int(row["stars"])

print("trues over all on training:", total_true/total)

trues over all on training: 0.41385


In [37]:
total = 0
total_true = 0
with open("../processed_files/data_stackoverflow_test.csv", "r", encoding="utf8") as f:
    reader = csv.DictReader(f, delimiter=",")
    for row in reader:
        total += 1
        total_true += int(row["stars"])

print("trues over all on test:", total_true/total)

trues over all on test: 0.41318


# New section